### 1.) Importing Essential Libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
mp_drawing = mp.solutions.drawing_utils # mp_drawing - visualizing the landmarks
mp_holistic = mp.solutions.holistic  # Mediapipe Solutions - - renders the 4 module pipeline
# Double - Arm Doumbell
counter=0
stage="Up"
# Overhead Press
press=0
stage_o="Up"
Counter=0
stage_p="Fall Detected"

### 2.) Function for Full Frame-0/1,Face - Front,Back,Left,Right,Pose - Straight,Down,inclined

In [2]:
def pose_estimate(pose_res,image):
    ##left_body_indices = [7,9,11,13,15,17,19,21,23,25,27,29,31]
    ##right_body_indices = [8,10,12,14,16,18,20,22,24,26,28,30,32]
    left_body_indices = [1,2,3,7,9,11,13,15,23,25,27,29,31]
    right_body_indices = [4,5,6,8,10,12,14,16,24,26,28,30,32]

    font = cv2.FONT_HERSHEY_SIMPLEX
    if pose_res !=None:
        x=pose_res.landmark # contains x,y,z,visibility coordinates
        count=0
        red=(0,0,255)
        green=(0,255,0)

        left_avg = 0
        right_avg = 0
        left_body_cnt = 0
        right_body_cnt = 0
        face_cnt = 0
        
        lsa = math.sqrt((x[11].x-x[27].x)**2+(x[11].y-x[27].y)**2)
        rsa = math.sqrt((x[12].x-x[28].x)**2+(x[12].y-x[28].y)**2)

        for k in range(10):
            if(x[k].visibility>0.5):
                face_cnt+=1

        for l in left_body_indices:
            left_avg+=x[l].visibility # left_avg = l + x[1].visibility -- adding all the left visibilty
            if(x[l].visibility>0.7):
                left_body_cnt+=1

        for r in right_body_indices:
            right_avg+=x[r].visibility
            if(x[r].visibility>0.7):
                right_body_cnt+=1

        left_avg /= len(left_body_indices) # Taking the avg of visibility
        right_avg /= len(right_body_indices)

        for y in x:
            if y.visibility>0.50:
                count+=1

        variation_avg = abs(left_avg - right_avg)
        variation_cnt = abs(left_body_cnt - right_body_cnt)

        pose = ""
        face = ""
        angle_toe_gnd = 0
        full_frame = 0

        if((variation_avg<=0.005)and(variation_cnt<=2)): ## Change made here (less than equal was modified) and threshold also changed
            if(face_cnt>7):
                face = "Front"
            else:
                face = "Back"

            if count >30:
                full_frame = 1
                if(abs(x[11].x-x[27].x)<=0.9*lsa)and(abs(x[12].x-x[28].x)<=0.9*rsa):
                    pose = "Straight"
            else:
                full_frame=0

        else:
            if(right_body_cnt>left_body_cnt) or (right_avg>left_avg):  ##checking left or right
                face = "Right"
                if right_body_cnt >= 10: ##Checking full body or not
                    full_frame =1
                    ang_to_check = math.degrees(math.atan(abs(x[12].y-x[30].y)/abs(x[12].x-x[30].x)))
                    if ang_to_check>85:
                        pose = "Straight"
                        angle_toe_gnd = ang_to_check
                    elif ang_to_check < 15:
                        pose = "Down"
                        angle_toe_gnd = math.degrees(math.atan(abs(x[12].y-x[32].y)/abs(x[12].x-x[32].x)))
                    else:
                        pose = "Inclined"
                        angle_toe_gnd = math.degrees(math.atan(abs(x[12].y-x[32].y)/abs(x[12].x-x[32].x)))
                else:
                    full_frame=0

            elif(left_body_cnt > right_body_cnt) or (right_avg < left_avg):  # checking left or right
                face = "Left"
                if left_body_cnt >= 10: ##Checking full body or not
                    full_frame = 1
                    ang_to_check = math.degrees(math.atan(abs(x[11].y-x[29].y)/abs(x[11].x-x[29].x)))
                    if ang_to_check > 85:
                        pose = "Straight"
                        angle_toe_gnd = ang_to_check
                    elif ang_to_check < 15:
                        pose = "Down"
                        angle_toe_gnd = math.degrees(math.atan(abs(x[11].y-x[31].y)/abs(x[11].x-x[31].x)))
                    else:
                        pose = "Inclined"
                        angle_toe_gnd = math.degrees(math.atan(abs(x[11].y-x[31].y)/abs(x[11].x-x[31].x)))
                else:
                    full_frame=0

    ang_knee_hip = math.degrees(math.atan(abs(x[23].y-x[25].y)/abs(x[23].x-x[25].x)))
    ang_wrist_shoulder = math.degrees(math.atan(abs(x[11].y-x[15].y)/abs(x[11].x-x[15].x)))

    hand_straight_angle = math.degrees(math.atan(abs(x[11].y-x[15].y)/abs(x[11].x-x[15].x)))
    elbow_straight_angle = math.degrees(math.atan(abs(x[14].y-x[16].y)/abs(x[14].x-x[16].x)))
    elbow_flat_angle = math.degrees(math.atan(abs(x[12].y-x[14].y)/abs(x[12].x-x[14].x)))

    return face,pose,full_frame,angle_toe_gnd,elbow_flat_angle,elbow_straight_angle,hand_straight_angle



### 3.1) Angle Function - Satisfies the Symmetric Problem - works for both left and right part

In [3]:
def calculate_angle(a,b,c,d,e,f,g,h,i):
    a = np.array(a)# First - Left
    b = np.array(b)# Mid - Left
    c = np.array(c)# End - Left
    d = np.array(d)# First - Right
    e = np.array(e)# Mid - Right
    f = np.array(f)# End - Right
    g = np.array(g)# End - left
    h = np.array(h)# End - left
    i = np.array(i)# End - left
    
    # 1- y, 0 - x
    # Left Part Arm
    radians_1 =np.arctan2(c[1]-b[1],c[0]-b[0]) -np.arctan2(a[1] -b[1],a[0]-b[0])
    angle_1 =np.abs(radians_1*180.0/np.pi) # rad to degree
    
    # Right Part Arm
    radians_2=np.arctan2(f[1]-e[1],f[0]-e[0]) -np.arctan2(d[1] -e[1],d[0]-e[0])
    angle_2=np.abs(radians_2*180.0/np.pi)
    
    # Left Part hip to ankle
    radians_3=np.arctan2(i[1]-h[1],i[0]-h[0]) -np.arctan2(g[1] -h[1],g[0]-h[0])
    angle_3=np.abs(radians_2*180.0/np.pi)
    
    if angle_1 >180:
        angle_1 = 360 -angle_1 
    
    if angle_2>180:
        angle_2=360-angle_2
    if angle_3>180:
        angle_3=360-angle_3
        
    return angle_1,angle_2,angle_3# when less then or equal to 180

### 3.2) Distance Function

In [4]:
# X-0,Y-1
def euclidian(point_1, point_2,point_3):
    dist_1=math.sqrt((point_1[0]-point_2[0])**2 + (point_1[1]-point_2[1])**2 )*100
    dist_2=math.sqrt((point_1[0]-point_3[0])**2 + (point_1[1]-point_3[1])**2 )*100
    return dist_1,dist_2

### 4.)Parameters Function to take the inputs from the Trainer

In [5]:
Signs =[]
def parameters():
        global cond_angle
        global Angle_para
        global Signs
        global condition_0
        global condition_1
        global save
        global t_dist
        global cond_dist
        global Dist_para
        save=[]
        t_dist=[]
        condition_0 = str(input("Angle or AngleGround or Distance or Angle&Distance:"))
        condition_1 = int(input("Enter the condition Number for the exercise you want to evaluate as 0,1,2.. :"))
        if condition_0 == "Angle":
                cond_angle= list(map(int, input("Points for which the angle is to be calculated: ").split()))
                Angle_para = list(map(int, input("Enter the Angle Parameters").split()))
                for i in range(0,2):
                    if i<2:
                        switcher ={0:"<",1:">"}
                        Signs.append(switcher.get(int(input("Enter the Sign's of Equality 0 for < & 1 for >: "))))  
                        save.append("Angle")
        if (condition_0 == 'Distance'):
            s = int(input('Number of distances to be calculated:'))
            for i in range(s):
                switcher = {
                    0: "normal",
                    1: "straight",
                    2: "perpendicular",
                }
                print('Type of distance: 0. normal     1. straight     2. perpendicular')
                cond_dist = switcher.get(int(input()), "nothing")
                if cond_dist == 'normal':
                    j, k = [int(s) for s in input('Points for the dist:').split(" ")]
                    temp = ('normal', j, k)
                    t_dist.append(temp)
                if cond_dist == 'straight':
                    j, k = [int(s) for s in input('Points for the first dist:').split(" ")]
                    l, m = [int(s) for s in input('Points for the second dist:').split(" ")]
                    temp = ('straight', j, k, l, m)
                    t_dist.append(temp)
                if cond_dist == 'perpendicular':
                    j, k = [int(s) for s in input('Points for the first dist:').split(" ")]
                    l, m = [int(s) for s in input('Points for the second dist:').split(" ")]
                    temp = ('perpendicular', j, k, l, m) 
                    t_dist.append(temp)
            save.append('Distance')
        
        if(condition_0=="Angle&Distance"):
                # Angle
                cond_angle= list(map(int, input("Points for which the angle is to be calculated: ").split()))
                Angle_para = list(map(int, input("Enter the Angle Parameters").split()))
                for i in range(0,2):
                    if i<2:
                        switcher ={0:"<=",1:">="}
                        Signs.append(switcher.get(int(input("Enter the Sign's of Equality 0 for <= & 1 for >=: "))))  
                        save.append("Angle&Distance")
                # Distance
                cond_dist = list(map(int,input("Points for which the distance is to be calculated: ").split()))
                Dist_para = list(map(int, input("Enter the Distance Parameters").split()))
                
                
        return condition_0,condition_1,cond_angle,Angle_para,Signs,t_dist,cond_dist,Dist_para
                
            
        
            
parameters()

     


Angle or AngleGround or Distance or Angle&Distance:Angle&Distance
Enter the condition Number for the exercise you want to evaluate as 0,1,2.. :2
Points for which the angle is to be calculated: 11 13 15 12 14 16 23 25 27 24 26 28
Enter the Angle Parameters70 84 90 104 170 182
Enter the Sign's of Equality 0 for <= & 1 for >=: 0
Enter the Sign's of Equality 0 for <= & 1 for >=: 1
Points for which the distance is to be calculated: 0 15 16
Enter the Distance Parameters10 20 13 22


('Angle&Distance',
 2,
 [11, 13, 15, 12, 14, 16, 23, 25, 27, 24, 26, 28],
 [70, 84, 90, 104, 170, 182],
 ['<=', '>='],
 [],
 [0, 15, 16],
 [10, 20, 13, 22])

# 5.) Curl-Counter Code for All Exercises

In [9]:
# VIDEO FEED - oNLY USING THE POSE MODULE OUT OF 4 MODULES IN m HOLISTIC MODULE
#color=(255,0,255)
Class = ['Single_arm_Dumbell','OH-Press','Plank','Neutral'] # Add on the Exercise names in this List
cap = cv2.VideoCapture(r"Plank_1.mp4")
## Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic: #
    while cap.isOpened():
        
        timer=cv2.getTickCount()
    
        ret,frame = cap.read()
        frame = cv2.resize(frame,(1280,720))

        #Recolor image
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # False as we can read as well as write w.r.t image
        
        # Make detections
        results =holistic.process(image)
        
        
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        if condition_0 == "Angle":
            if condition_1 == 0:
                    try: 
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                            shoulder_1 = [landmarks[cond_angle[0]].x,landmarks[cond_angle[0]].y]
                            elbow_1 = [landmarks[cond_angle[1]].x,landmarks[cond_angle[1]].y]
                            wrist_1 = [landmarks[cond_angle[2]].x,landmarks[cond_angle[2]].y]
                            
                            shoulder_2 = [landmarks[12].x,landmarks[12].y]
                            elbow_2 = [landmarks[14].x,landmarks[14].y]
                            wrist_2 = [landmarks[16].x,landmarks[16].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)
                            
                            per = np.interp(angle[0],(20,160),(100,0)) # LEFT
                            bar = np.interp(angle[0],(30,160),(600,200))
            
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            #print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Curl Counter Logic
                            color =(255,0,255)
                            if Signs[0]=="<" or Signs[1]==">":
                                if angle[0]>Angle_para[1]:
                                    stage = "Down"
                                    if per ==0:
                                        color =(0,255,0)
                                if angle[0]<Angle_para[0] and stage =="Down": # Count when we moves from down to up , >160 is full flat so down
                                    stage="Up"
                                    if per ==100:
                                        color=(0,255,0)
                                    counter+=1
                                    print(counter)
                            
                            
                            
            
                    except:
                         pass
                            
                            # Render curl counter and status box
                    cv2.rectangle(image,(5,5),(270,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
                            # Draw Bar
                    cv2.rectangle(image,(1000,200),(1055,600),color,3)
                    cv2.rectangle(image,(1000,int(bar)),(1055,600),color,cv2.FILLED)
                    cv2.putText(image,f'{int(per)}%',(1000,170),cv2.FONT_HERSHEY_PLAIN, 2,color, 3)
        
       
                            # Rep data
                    cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,str(counter),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                    cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,stage,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                    cv2.putText(image,Class[0],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                    if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                    if full_frame_res == 0:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 255, 0), 10) # green
                    else:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 0 ,255), 10) #red
                                    
    
            
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                            break
            elif condition_1==1:
                    try: 
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                            shoulder_1 = [landmarks[cond_angle[0]].x,landmarks[cond_angle[0]].y]
                            elbow_1 = [landmarks[cond_angle[1]].x,landmarks[cond_angle[1]].y]
                            wrist_1 = [landmarks[cond_angle[2]].x,landmarks[cond_angle[2]].y]
                            
                            shoulder_2 = [landmarks[cond_angle[3]].x,landmarks[cond_angle[3]].y]
                            elbow_2 = [landmarks[cond_angle[4]].x,landmarks[cond_angle[4]].y]
                            wrist_2 = [landmarks[cond_angle[5]].x,landmarks[cond_angle[5]].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)
                            
                            #per = np.interp(angle[0],(20,160),(100,0)) # LEFT
                            #bar = np.interp(angle[0],(30,160),(600,200))
            
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [800, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_1, [800, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            #print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Curl Counter Logic
                            color =(255,0,255)
                            if Signs[0]=="<" and Signs[1]==">":
                                # 2.) Oh- Press Curl Counter    
                                    if angle[0]<70:
                                           stage_o = "Down"
                                    if angle[0]>110 and stage_o=="Down" and angle[0] is not None:
                                            stage_o="Up"
                                            
                                            press+=1
                                            print(press)
                            
                    except:
                         pass
                            
                            # Render curl counter and status box
                    cv2.rectangle(image,(5,5),(270,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
                            # Draw Bar
                    #cv2.rectangle(image,(1000,200),(1055,600),color,3)
                    #cv2.rectangle(image,(1000,int(bar)),(1055,600),color,cv2.FILLED)
                    #cv2.putText(image,f'{int(per)}%',(1000,170),cv2.FONT_HERSHEY_PLAIN, 2,color, 3)
        
       
                            # Rep data
                    cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,str(press),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                    cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,stage_o,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                    cv2.putText(image,Class[1],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                    if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                                    if full_frame_res == 0:
                                          cv2.rectangle(image, (0,0), (800,720), (0, 255, 0), 10) # green
                                    else:
                                          cv2.rectangle(image, (0,0), (800,720), (0, 0 ,255), 10) #red
            
            
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                            break
                            
        
        if condition_0 == "Angle&Distance":
            if condition_1 == 2:
                try:
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                           
                            #2-Distance
                            point_1=[landmarks[cond_dist[0]].x,landmarks[cond_dist[0]].y]
                            point_2=[landmarks[cond_dist[1]].x,landmarks[cond_dist[1]].y]
                            point_3=[landmarks[cond_dist[2]].x,landmarks[cond_dist[2]].y]
                            
                            # 3 Angle's
                            shoulder_1 = [landmarks[11].x,landmarks[11].y]
                            elbow_1 = [landmarks[13].x,landmarks[13].y]
                            wrist_1 = [landmarks[15].x,landmarks[15].y]
                            
                            shoulder_2 = [landmarks[12].x,landmarks[12].y]
                            elbow_2 = [landmarks[14].x,landmarks[14].y]
                            wrist_2 = [landmarks[16].x,landmarks[16].y]
                            
                            hip_1=[landmarks[23].x,landmarks[23].y] # Left Part
                            knee_1=[landmarks[25].x,landmarks[25].y]
                            ankle_1=[landmarks[27].x,landmarks[27].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2,hip_1,knee_1,ankle_1)
                            dist = euclidian(point_1,point_2,point_3)
                            
                            # Angle - left arm
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            # Angle - Right Arm
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_2, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            # Dist Left -1
                            cv2.putText(image, str(round(dist[0],1)), 
                            tuple(np.multiply(hip_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            # Angle - left hip
                            cv2.putText(image, str(round(angle[2],1)), 
                            tuple(np.multiply(knee_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            #print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Plank up and Fall Logic
                            if Signs[0]=="<=" and Signs[1]==">=":
                                if ((70<=angle[0]<=84 and 90<=angle[1]<=104) and (Dist_para[0]<=dist[0]<=Dist_para[1] or Dist_para[0]<=Dist_para[1])):
                                    stage_p="Up Side"
                                    Counter=1
                                elif ((Dist_para[2]<=dist[0]<=Dist_para[3] or Dist_para[2]<=dist[1]<=Dist_para[3]) and (170<=angle[0]<=182 and 170<=angle[1]<=182)):
                                    cv2.putText(image,Class[2]+"-Full Frame not detected",(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
                                    Counter=1
                                    stage_p ="Up Front"
                                else:
                                    stage_p="Fall Detected"
                                    Counter=0
                        
                except:
                    pass
                                    
                            # Render curl counter and status box
                cv2.rectangle(image,(5,5),(420,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)

        
       
                            # Rep data
                cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image,str(Counter),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image,stage_p,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                cv2.putText(image,Class[2],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                                    if full_frame_res == 0:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 0, 255), 10) # green
                                    else:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 255,0), 10) #red
                                    
    
            
                cv2.imshow("Mediapipe Feed",image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                     break    
            
            
            else:
                    cv2.putText(image,Class[3],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                           break
        cv2.imwrite(r"C:\Users\KUNAL\Documents\EvolvFit."+str(id)+".jpg",frame)
        output.write(frame)
                
       
                  
    cap.release()
    cv2.destroyAllWindows()
                    

[x: 0.2037934809923172
y: 0.7773364782333374
z: -0.0894521102309227
visibility: 0.9992990493774414
, x: 0.2026110142469406
y: 0.7566341161727905
z: -0.1140957847237587
visibility: 0.999210000038147
, x: 0.20448124408721924
y: 0.7541071772575378
z: -0.11410652101039886
visibility: 0.9992102384567261
, x: 0.2065620869398117
y: 0.7513520121574402
z: -0.11404617130756378
visibility: 0.9993120431900024
, x: 0.2022899091243744
y: 0.756853461265564
z: -0.07317081838846207
visibility: 0.9989156723022461
, x: 0.2038867175579071
y: 0.7547029852867126
z: -0.07322608679533005
visibility: 0.9988619089126587
, x: 0.20561406016349792
y: 0.7522966861724854
z: -0.07309488952159882
visibility: 0.999113142490387
, x: 0.22467216849327087
y: 0.7387391924858093
z: -0.1768084317445755
visibility: 0.998872697353363
, x: 0.22260844707489014
y: 0.7364056706428528
z: 0.003696468425914645
visibility: 0.9980732202529907
, x: 0.22122910618782043
y: 0.7831897139549255
z: -0.10839112102985382
visibility: 0.9991858601

[x: 0.20328886806964874
y: 0.7766446471214294
z: -0.10261012613773346
visibility: 0.9993485808372498
, x: 0.20236916840076447
y: 0.7559718489646912
z: -0.12713465094566345
visibility: 0.999276876449585
, x: 0.20428256690502167
y: 0.7534297704696655
z: -0.12715363502502441
visibility: 0.9992676377296448
, x: 0.20640632510185242
y: 0.7507064938545227
z: -0.12710131704807281
visibility: 0.9993722438812256
, x: 0.20197081565856934
y: 0.7563470602035522
z: -0.0874185860157013
visibility: 0.9989981055259705
, x: 0.20356610417366028
y: 0.7542505264282227
z: -0.08747528493404388
visibility: 0.9989351034164429
, x: 0.20529720187187195
y: 0.7519515752792358
z: -0.08734817802906036
visibility: 0.9991812705993652
, x: 0.2245243787765503
y: 0.7379764914512634
z: -0.18806853890419006
visibility: 0.9989610910415649
, x: 0.22230830788612366
y: 0.7361189126968384
z: -0.012731358408927917
visibility: 0.9982141852378845
, x: 0.22089478373527527
y: 0.7818728685379028
z: -0.12049510329961777
visibility: 0.

[x: 0.20307795703411102
y: 0.7762228846549988
z: -0.10366328060626984
visibility: 0.9993961453437805
, x: 0.20223203301429749
y: 0.7555630207061768
z: -0.1282513439655304
visibility: 0.9993410706520081
, x: 0.20415882766246796
y: 0.7530023455619812
z: -0.12826964259147644
visibility: 0.9993234872817993
, x: 0.20629680156707764
y: 0.7502762675285339
z: -0.12821760773658752
visibility: 0.9994300603866577
, x: 0.2018042355775833
y: 0.7560211420059204
z: -0.08832177519798279
visibility: 0.9990766048431396
, x: 0.20339593291282654
y: 0.7539478540420532
z: -0.08837859332561493
visibility: 0.9990054965019226
, x: 0.2051231563091278
y: 0.7517004013061523
z: -0.08825093507766724
visibility: 0.9992458820343018
, x: 0.22438661754131317
y: 0.7373974919319153
z: -0.18942296504974365
visibility: 0.9990475177764893
, x: 0.22210398316383362
y: 0.7358611226081848
z: -0.013247149996459484
visibility: 0.9983481764793396
, x: 0.22069165110588074
y: 0.7812126874923706
z: -0.12168492376804352
visibility: 0.

[x: 0.20298628509044647
y: 0.7760397791862488
z: -0.10257571935653687
visibility: 0.9994262456893921
, x: 0.2021711766719818
y: 0.7553401589393616
z: -0.12717373669147491
visibility: 0.9993818402290344
, x: 0.2041027545928955
y: 0.7527802586555481
z: -0.12719157338142395
visibility: 0.9993598461151123
, x: 0.2062474489212036
y: 0.7500530481338501
z: -0.1271388828754425
visibility: 0.9994670748710632
, x: 0.2017463743686676
y: 0.7558152079582214
z: -0.08714587241411209
visibility: 0.9991248846054077
, x: 0.20334291458129883
y: 0.7537540793418884
z: -0.08720212429761887
visibility: 0.9990488290786743
, x: 0.20507463812828064
y: 0.751533567905426
z: -0.08707396686077118
visibility: 0.9992843866348267
, x: 0.2243576943874359
y: 0.7371553778648376
z: -0.18860380351543427
visibility: 0.9991053342819214
, x: 0.2220526486635208
y: 0.7357468605041504
z: -0.011960216797888279
visibility: 0.9984282851219177
, x: 0.22064778208732605
y: 0.7810050845146179
z: -0.12071911245584488
visibility: 0.99939

[x: 0.2027101069688797
y: 0.7759593725204468
z: -0.1059221476316452
visibility: 0.9994490742683411
, x: 0.20199617743492126
y: 0.7552202939987183
z: -0.13068442046642303
visibility: 0.9994110465049744
, x: 0.20393387973308563
y: 0.7526708841323853
z: -0.1307019740343094
visibility: 0.9993880987167358
, x: 0.20608749985694885
y: 0.7499564290046692
z: -0.13065418601036072
visibility: 0.9994952082633972
, x: 0.20159557461738586
y: 0.755676805973053
z: -0.08993230015039444
visibility: 0.9991557598114014
, x: 0.20321080088615417
y: 0.7536152601242065
z: -0.0899864062666893
visibility: 0.9990766644477844
, x: 0.20496006309986115
y: 0.7514017820358276
z: -0.08985462039709091
visibility: 0.9993072748184204
, x: 0.22426649928092957
y: 0.7371068596839905
z: -0.19245050847530365
visibility: 0.9991519451141357
, x: 0.2219594269990921
y: 0.7357079386711121
z: -0.012615546584129333
visibility: 0.9984726905822754
, x: 0.22042417526245117
y: 0.7809771299362183
z: -0.12424297630786896
visibility: 0.999

...

[x: 0.5002009272575378
y: 0.660790205001831
z: -0.6510982513427734
visibility: 0.9994919896125793
, x: 0.5096544027328491
y: 0.6383810043334961
z: -0.6571103930473328
visibility: 0.9995467066764832
, x: 0.5161851644515991
y: 0.6340900659561157
z: -0.6570316553115845
visibility: 0.9991027116775513
, x: 0.5227421522140503
y: 0.6297280788421631
z: -0.6569926738739014
visibility: 0.9995673894882202
, x: 0.4907083511352539
y: 0.639904797077179
z: -0.6572988629341125
visibility: 0.9995611310005188
, x: 0.48348695039749146
y: 0.6362003684043884
z: -0.6573341488838196
visibility: 0.9992819428443909
, x: 0.475241094827652
y: 0.6320516467094421
z: -0.657292366027832
visibility: 0.9996605515480042
, x: 0.5340785384178162
y: 0.5967172980308533
z: -0.5702487230300903
visibility: 0.997776210308075
, x: 0.46301573514938354
y: 0.6001956462860107
z: -0.5715087652206421
visibility: 0.9986535310745239
, x: 0.5106863379478455
y: 0.6496094465255737
z: -0.6077511310577393
visibility: 0.9984955787658691
, x:

[x: 0.5001111626625061
y: 0.6628184914588928
z: -0.6600823402404785
visibility: 0.9994683265686035
, x: 0.5097074508666992
y: 0.6396739482879639
z: -0.6654639840126038
visibility: 0.999506950378418
, x: 0.5162343978881836
y: 0.6352165341377258
z: -0.6653872728347778
visibility: 0.9990416169166565
, x: 0.5227600336074829
y: 0.6308746933937073
z: -0.665350615978241
visibility: 0.9995280504226685
, x: 0.49082884192466736
y: 0.6414871215820312
z: -0.6657899618148804
visibility: 0.9995237588882446
, x: 0.4837206304073334
y: 0.637712836265564
z: -0.6658294200897217
visibility: 0.9992309808731079
, x: 0.47549542784690857
y: 0.633192241191864
z: -0.6657870411872864
visibility: 0.9996310472488403
, x: 0.5341539978981018
y: 0.5978385210037231
z: -0.5761400461196899
visibility: 0.9976034760475159
, x: 0.46302351355552673
y: 0.6005691289901733
z: -0.5783290266990662
visibility: 0.9985545873641968
, x: 0.5106483697891235
y: 0.651788055896759
z: -0.6159578561782837
visibility: 0.9983826875686646
, x

[x: 0.4998864531517029
y: 0.6632519364356995
z: -0.6493626832962036
visibility: 0.999458909034729
, x: 0.5095916986465454
y: 0.6402937769889832
z: -0.6569674015045166
visibility: 0.999485194683075
, x: 0.5160778760910034
y: 0.6359043121337891
z: -0.6568858623504639
visibility: 0.9990274310112
, x: 0.5225237011909485
y: 0.6316919326782227
z: -0.6568478345870972
visibility: 0.9995105862617493
, x: 0.49081382155418396
y: 0.6421282291412354
z: -0.6566919684410095
visibility: 0.9995008111000061
, x: 0.48375892639160156
y: 0.6384254693984985
z: -0.6567220687866211
visibility: 0.9992098808288574
, x: 0.47554898262023926
y: 0.6339508295059204
z: -0.6566653847694397
visibility: 0.9996120929718018
, x: 0.5340673327445984
y: 0.5986941456794739
z: -0.5746188759803772
visibility: 0.9975456595420837
, x: 0.4630238115787506
y: 0.6009867191314697
z: -0.5739844441413879
visibility: 0.9984948039054871
, x: 0.5104600787162781
y: 0.6521017551422119
z: -0.6075742840766907
visibility: 0.9983220100402832
, x

[x: 0.4995303750038147
y: 0.6640677452087402
z: -0.6449583768844604
visibility: 0.9994316697120667
, x: 0.5095793604850769
y: 0.6409593224525452
z: -0.6523975133895874
visibility: 0.9994407892227173
, x: 0.5161197781562805
y: 0.6364800930023193
z: -0.6523159146308899
visibility: 0.9989719390869141
, x: 0.5225863456726074
y: 0.6322465538978577
z: -0.652280330657959
visibility: 0.9994727969169617
, x: 0.4906178414821625
y: 0.6430670619010925
z: -0.6523273587226868
visibility: 0.9994587898254395
, x: 0.4835941791534424
y: 0.6393447518348694
z: -0.6523586511611938
visibility: 0.9991600513458252
, x: 0.4754815101623535
y: 0.6348956823348999
z: -0.6523051261901855
visibility: 0.9995798468589783
, x: 0.5342319011688232
y: 0.5990457534790039
z: -0.5709692239761353
visibility: 0.9974067211151123
, x: 0.46301859617233276
y: 0.6011547446250916
z: -0.5703838467597961
visibility: 0.9983872175216675
, x: 0.5103973150253296
y: 0.6515461802482605
z: -0.6031225323677063
visibility: 0.9982455968856812
,

[x: 0.49955010414123535
y: 0.6644159555435181
z: -0.6449174880981445
visibility: 0.9994159936904907
, x: 0.5098099112510681
y: 0.6411101818084717
z: -0.6526334285736084
visibility: 0.9994192123413086
, x: 0.5164262056350708
y: 0.6366242170333862
z: -0.6525447368621826
visibility: 0.998935341835022
, x: 0.5229355096817017
y: 0.6323938369750977
z: -0.6525071859359741
visibility: 0.9994548559188843
, x: 0.4906781315803528
y: 0.6432406902313232
z: -0.6528485417366028
visibility: 0.9994392395019531
, x: 0.48364895582199097
y: 0.639510989189148
z: -0.6528802514076233
visibility: 0.999130368232727
, x: 0.47561392188072205
y: 0.6350566148757935
z: -0.6528328061103821
visibility: 0.9995655417442322
, x: 0.5346091985702515
y: 0.5991355180740356
z: -0.5707187056541443
visibility: 0.997338056564331
, x: 0.46306705474853516
y: 0.6011961102485657
z: -0.5715813636779785
visibility: 0.9983299374580383
, x: 0.5106024742126465
y: 0.6517536044120789
z: -0.6028991341590881
visibility: 0.998239278793335
, 

# Rough Part

In [63]:
stage_p="Fall"
Counter=0
Class = ['Single_arm_Dumbell','OH-Press','Neutral','Plank'] # Add on the Exercise names in this List
cap = cv2.VideoCapture(r"Plank_1.mp4")
## Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic: #
    while cap.isOpened():
        timer=cv2.getTickCount()
        ret,frame = cap.read()
        frame = cv2.resize(frame,(1280,720))
        
        #Recolor image
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # False as we can read as well as write w.r.t image
        
        # Make detections
        results =holistic.process(image)
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        try: 
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                           
                            #2-Distance
                            point_1=[landmarks[0].x,landmarks[0].y]
                            point_2=[landmarks[15].x,landmarks[15].y]
                            point_3=[landmarks[16].x,landmarks[16].y]
                            
                            # 3 Angle's
                            shoulder_1 = [landmarks[11].x,landmarks[11].y]
                            elbow_1 = [landmarks[13].x,landmarks[13].y]
                            wrist_1 = [landmarks[15].x,landmarks[15].y]
                            
                            shoulder_2 = [landmarks[12].x,landmarks[12].y]
                            elbow_2 = [landmarks[14].x,landmarks[14].y]
                            wrist_2 = [landmarks[16].x,landmarks[16].y]
                            
                            hip_1=[landmarks[23].x,landmarks[23].y] # Left Part
                            knee_1=[landmarks[25].x,landmarks[25].y]
                            ankle_1=[landmarks[27].x,landmarks[27].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2,hip_1,knee_1,ankle_1)
                            dist = euclidian(point_1,point_2,point_3)
                            
                            # Angle - left arm
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            # Angle - Right Arm
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_2, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            # Dist Left -1
                            cv2.putText(image, str(round(dist[0],2)), 
                            tuple(np.multiply(hip_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            # Angle - left hip
                            cv2.putText(image, str(round(angle[2],2)), 
                            tuple(np.multiply(knee_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            #print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Plank up and Fall Logic
                            
                            if ((70<=angle[0]<=78 and 90<=angle[1]<=100) and (10<=dist[0]<=18 or 10<=dist[1]<=18)):
                                stage_p="Up_Side"
                                Counter=1
                            elif ((13<=dist[0]<=22 or 13<=dist[1]<=22) and (170<=angle[0]<=182 and 170<=angle[1]<=182)):
                                cv2.putText(image,Class[3]+"-Full Frame not detected",(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
                                Counter=1
                                stage_p ="Up_Front"
                            else:
                                stage_p="Fall Detected"
                                Counter=0
                            
                           
                            
        except:
            pass
                            
                            # Render curl counter and status box
        cv2.rectangle(image,(5,5),(400,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)

        
       
                            # Rep data
        cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image,str(Counter),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
        cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image,stage_p,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
        cv2.putText(image,Class[3],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
        if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                                    if full_frame_res == 0:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 0, 255), 10) # green
                                    else:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 255,0), 10) #red
                                    
    
            
        cv2.imshow("Mediapipe Feed",image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
             break
    cap.release()
    cv2.destroyAllWindows()

[x: 0.2037934809923172
y: 0.7773364782333374
z: -0.0894521102309227
visibility: 0.9992990493774414
, x: 0.2026110142469406
y: 0.7566341161727905
z: -0.1140957847237587
visibility: 0.999210000038147
, x: 0.20448124408721924
y: 0.7541071772575378
z: -0.11410652101039886
visibility: 0.9992102384567261
, x: 0.2065620869398117
y: 0.7513520121574402
z: -0.11404617130756378
visibility: 0.9993120431900024
, x: 0.2022899091243744
y: 0.756853461265564
z: -0.07317081838846207
visibility: 0.9989156723022461
, x: 0.2038867175579071
y: 0.7547029852867126
z: -0.07322608679533005
visibility: 0.9988619089126587
, x: 0.20561406016349792
y: 0.7522966861724854
z: -0.07309488952159882
visibility: 0.999113142490387
, x: 0.22467216849327087
y: 0.7387391924858093
z: -0.1768084317445755
visibility: 0.998872697353363
, x: 0.22260844707489014
y: 0.7364056706428528
z: 0.003696468425914645
visibility: 0.9980732202529907
, x: 0.22122910618782043
y: 0.7831897139549255
z: -0.10839112102985382
visibility: 0.9991858601

[x: 0.20328886806964874
y: 0.7766446471214294
z: -0.10261012613773346
visibility: 0.9993485808372498
, x: 0.20236916840076447
y: 0.7559718489646912
z: -0.12713465094566345
visibility: 0.999276876449585
, x: 0.20428256690502167
y: 0.7534297704696655
z: -0.12715363502502441
visibility: 0.9992676377296448
, x: 0.20640632510185242
y: 0.7507064938545227
z: -0.12710131704807281
visibility: 0.9993722438812256
, x: 0.20197081565856934
y: 0.7563470602035522
z: -0.0874185860157013
visibility: 0.9989981055259705
, x: 0.20356610417366028
y: 0.7542505264282227
z: -0.08747528493404388
visibility: 0.9989351034164429
, x: 0.20529720187187195
y: 0.7519515752792358
z: -0.08734817802906036
visibility: 0.9991812705993652
, x: 0.2245243787765503
y: 0.7379764914512634
z: -0.18806853890419006
visibility: 0.9989610910415649
, x: 0.22230830788612366
y: 0.7361189126968384
z: -0.012731358408927917
visibility: 0.9982141852378845
, x: 0.22089478373527527
y: 0.7818728685379028
z: -0.12049510329961777
visibility: 0.

[x: 0.20307795703411102
y: 0.7762228846549988
z: -0.10366328060626984
visibility: 0.9993961453437805
, x: 0.20223203301429749
y: 0.7555630207061768
z: -0.1282513439655304
visibility: 0.9993410706520081
, x: 0.20415882766246796
y: 0.7530023455619812
z: -0.12826964259147644
visibility: 0.9993234872817993
, x: 0.20629680156707764
y: 0.7502762675285339
z: -0.12821760773658752
visibility: 0.9994300603866577
, x: 0.2018042355775833
y: 0.7560211420059204
z: -0.08832177519798279
visibility: 0.9990766048431396
, x: 0.20339593291282654
y: 0.7539478540420532
z: -0.08837859332561493
visibility: 0.9990054965019226
, x: 0.2051231563091278
y: 0.7517004013061523
z: -0.08825093507766724
visibility: 0.9992458820343018
, x: 0.22438661754131317
y: 0.7373974919319153
z: -0.18942296504974365
visibility: 0.9990475177764893
, x: 0.22210398316383362
y: 0.7358611226081848
z: -0.013247149996459484
visibility: 0.9983481764793396
, x: 0.22069165110588074
y: 0.7812126874923706
z: -0.12168492376804352
visibility: 0.

[x: 0.20298628509044647
y: 0.7760397791862488
z: -0.10257571935653687
visibility: 0.9994262456893921
, x: 0.2021711766719818
y: 0.7553401589393616
z: -0.12717373669147491
visibility: 0.9993818402290344
, x: 0.2041027545928955
y: 0.7527802586555481
z: -0.12719157338142395
visibility: 0.9993598461151123
, x: 0.2062474489212036
y: 0.7500530481338501
z: -0.1271388828754425
visibility: 0.9994670748710632
, x: 0.2017463743686676
y: 0.7558152079582214
z: -0.08714587241411209
visibility: 0.9991248846054077
, x: 0.20334291458129883
y: 0.7537540793418884
z: -0.08720212429761887
visibility: 0.9990488290786743
, x: 0.20507463812828064
y: 0.751533567905426
z: -0.08707396686077118
visibility: 0.9992843866348267
, x: 0.2243576943874359
y: 0.7371553778648376
z: -0.18860380351543427
visibility: 0.9991053342819214
, x: 0.2220526486635208
y: 0.7357468605041504
z: -0.011960216797888279
visibility: 0.9984282851219177
, x: 0.22064778208732605
y: 0.7810050845146179
z: -0.12071911245584488
visibility: 0.99939

[x: 0.2027101069688797
y: 0.7759593725204468
z: -0.1059221476316452
visibility: 0.9994490742683411
, x: 0.20199617743492126
y: 0.7552202939987183
z: -0.13068442046642303
visibility: 0.9994110465049744
, x: 0.20393387973308563
y: 0.7526708841323853
z: -0.1307019740343094
visibility: 0.9993880987167358
, x: 0.20608749985694885
y: 0.7499564290046692
z: -0.13065418601036072
visibility: 0.9994952082633972
, x: 0.20159557461738586
y: 0.755676805973053
z: -0.08993230015039444
visibility: 0.9991557598114014
, x: 0.20321080088615417
y: 0.7536152601242065
z: -0.0899864062666893
visibility: 0.9990766644477844
, x: 0.20496006309986115
y: 0.7514017820358276
z: -0.08985462039709091
visibility: 0.9993072748184204
, x: 0.22426649928092957
y: 0.7371068596839905
z: -0.19245050847530365
visibility: 0.9991519451141357
, x: 0.2219594269990921
y: 0.7357079386711121
z: -0.012615546584129333
visibility: 0.9984726905822754
, x: 0.22042417526245117
y: 0.7809771299362183
z: -0.12424297630786896
visibility: 0.999

[x: 0.2025555670261383
y: 0.7758744955062866
z: -0.10750539600849152
visibility: 0.9994643926620483
, x: 0.20187227427959442
y: 0.7551173567771912
z: -0.1321462243795395
visibility: 0.9994272589683533
, x: 0.2038138210773468
y: 0.7525581121444702
z: -0.13216522336006165
visibility: 0.9994068741798401
, x: 0.20597246289253235
y: 0.749832808971405
z: -0.13211776316165924
visibility: 0.9995112419128418
, x: 0.20147529244422913
y: 0.7555815577507019
z: -0.09157875180244446
visibility: 0.9991706609725952
, x: 0.20309975743293762
y: 0.7535132765769958
z: -0.09163161367177963
visibility: 0.999094545841217
, x: 0.20485885441303253
y: 0.7512853145599365
z: -0.0914977490901947
visibility: 0.9993188381195068
, x: 0.22418563067913055
y: 0.7369455099105835
z: -0.193487286567688
visibility: 0.9991769194602966
, x: 0.22189617156982422
y: 0.7355916500091553
z: -0.014278861694037914
visibility: 0.9984928369522095
, x: 0.2202763855457306
y: 0.7809034585952759
z: -0.12565375864505768
visibility: 0.999451

[x: 0.20248475670814514
y: 0.7758012413978577
z: -0.10600074380636215
visibility: 0.9994808435440063
, x: 0.20183055102825165
y: 0.7550021409988403
z: -0.13045695424079895
visibility: 0.999447226524353
, x: 0.20377835631370544
y: 0.7524273991584778
z: -0.1304756999015808
visibility: 0.9994262456893921
, x: 0.20594342052936554
y: 0.749683141708374
z: -0.13042831420898438
visibility: 0.9995297789573669
, x: 0.20142468810081482
y: 0.7554829120635986
z: -0.09021525084972382
visibility: 0.9991973042488098
, x: 0.20305077731609344
y: 0.7534117102622986
z: -0.09026841819286346
visibility: 0.9991191029548645
, x: 0.2048121988773346
y: 0.7511712908744812
z: -0.09013549983501434
visibility: 0.9993403553962708
, x: 0.224161297082901
y: 0.7367390990257263
z: -0.19161458313465118
visibility: 0.9992029666900635
, x: 0.22186660766601562
y: 0.7354608178138733
z: -0.013702723197638988
visibility: 0.9985277056694031
, x: 0.22021996974945068
y: 0.780867338180542
z: -0.12411491572856903
visibility: 0.9994

[x: 0.20240621268749237
y: 0.7757604122161865
z: -0.10565896332263947
visibility: 0.9994972944259644
, x: 0.2017991840839386
y: 0.754910945892334
z: -0.13012665510177612
visibility: 0.9994672536849976
, x: 0.20375213027000427
y: 0.7523119449615479
z: -0.13014517724514008
visibility: 0.9994477033615112
, x: 0.20591585338115692
y: 0.7495409846305847
z: -0.1300964206457138
visibility: 0.9995478391647339
, x: 0.20138008892536163
y: 0.7554172277450562
z: -0.08985931426286697
visibility: 0.9992226958274841
, x: 0.20300836861133575
y: 0.75334632396698
z: -0.08991079777479172
visibility: 0.9991458058357239
, x: 0.20477262139320374
y: 0.751102864742279
z: -0.08977913856506348
visibility: 0.9993605613708496
, x: 0.22418248653411865
y: 0.7364662885665894
z: -0.1914590448141098
visibility: 0.9992307424545288
, x: 0.2218445986509323
y: 0.7353178858757019
z: -0.013568677008152008
visibility: 0.9985687732696533
, x: 0.22019203007221222
y: 0.7808181643486023
z: -0.12386409938335419
visibility: 0.99949

In [46]:
c=0
a= [10.2,20]
if 0 <= a[0]<=10.8:
    c=1
    print(c)
else:
    print("we")


1


In [36]:
a[0]

10.9

In [23]:
shoulder_1 = [landmarks[11].x,landmarks[11].y]
elbow_1 = [landmarks[13].x,landmarks[13].y]
wrist_1 = [landmarks[15].x,landmarks[15].y]
shoulder_2 = [landmarks[12].x,landmarks[12].y]
elbow_2 = [landmarks[14].x,landmarks[14].y]
wrist_2 = [landmarks[16].x,landmarks[16].y]
hip_1=[landmarks[23].x,landmarks[23].y] # Left Part
knee_1=[landmarks[25].x,landmarks[25].y]
ankle_1=[landmarks[27].x,landmarks[27].y]


In [24]:
angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2,hip_1,knee_1,ankle_1)
angle

(75.22770650803577, 94.34172452886139, 94.34172452886139)

In [22]:
point_1=[landmarks[0].x,landmarks[0].y]
point_2=[landmarks[15].x,landmarks[15].y]
point_3=[landmarks[16].x,landmarks[16].y]
dist = euclidian(point_1,point_2,point_3)
dist

(14.569217732908553, 11.555979420732603)

In [10]:
counter=0
stage ="Up"
def single_arm(landmarks,image):
                    global counter
                    global stage
                            
                    try: 
                            
                            shoulder_1 = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
                            elbow_1 = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
                            wrist_1 = [landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
                            
                            shoulder_2 = [landmarks[12].x,landmarks[12].y]
                            elbow_2 = [landmarks[14].x,landmarks[14].y]
                            wrist_2 = [landmarks[16].x,landmarks[16].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)
                            
                            per = np.interp(angle[0],(20,160),(100,0)) # LEFT
                            bar = np.interp(angle[0],(30,160),(600,200))
            
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Curl Counter Logic
                            color =(255,0,255)
                            if angle[0]>160:
                                stage = "Down"
                                if per ==0:
                                    color =(0,255,0)
                            if angle[0]<20 and stage =="Down": # Count when we moves from down to up , >160 is full flat so down
                                stage="Up"
                                if per ==100:
                                    color=(0,255,0)
                                counter+=1
                                print(counter)
                            
            
                    except:
                         pass
                            
                            # Render curl counter and status box
                    cv2.rectangle(image,(5,5),(270,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
                            # Draw Bar
                    cv2.rectangle(image,(1000,200),(1055,600),color,3)
                    cv2.rectangle(image,(1000,int(bar)),(1055,600),color,cv2.FILLED)
                    cv2.putText(image,f'{int(per)}%',(1000,170),cv2.FONT_HERSHEY_PLAIN, 2,color, 3)
        
       
                            # Rep data
                    cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,str(counter),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                    cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,stage,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                    cv2.putText(image,Class[0],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                    if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                    if full_frame_res == 1:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 255, 0), 10) # green
                    else:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 0 ,255), 10) #red
                                    
    
                                   
        
                                    

In [58]:
elbow_1 = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
elbow

[0.546816349029541, 0.6440072655677795]

In [66]:
shoulder_1,elbow_1,wrist_1,shoulder_2,elbow_2,wrist_2

([0.6984878182411194, 0.42096492648124695],
 [0.737403392791748, 0.6836262345314026],
 [0.7224488854408264, 0.9301980137825012],
 [0.5472438931465149, 0.40640661120414734],
 [0.5466867685317993, 0.665658175945282],
 [0.5328742265701294, 0.8664588332176208])

In [22]:
angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)

NameError: name 'shoulder_2' is not defined

In [69]:
tuple(np.multiply(elbow_1, [1280, 720]).astype(int))

(943, 492)

In [ ]:
Starting Condition - Distance, Angle, Angle- Ground
Distance Type: Normal, Straight,Perpendicular
insert points
def parameter for all para input
Angle Type: points for which angle needs to be calculated 
Enter the angle parameters: single line multiple inputs
if angle then < -0 >-1 enter user

In [17]:
a = int(input("Enter a Number: \n"))
para =[a,b]

Enter a Number: 
1
Enter a Number: 
2


In [44]:
#a,b = input("Enter: ").split()
b
x = list(map(int, input("Enter a multiple value: ").split()))

Enter a multiple value: 11 12 12


In [6]:
# VIDEO FEED - oNLY USING THE POSE MODULE OUT OF 4 MODULES IN m HOLISTIC MODULE
#color=(255,0,255)
Class = ['Single_arm_Dumbell','OH-Press','Neutral'] # Add on the Exercise names in this List
cap = cv2.VideoCapture(r"curls.mp4")
## Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic: #
    while cap.isOpened():
        timer=cv2.getTickCount()
        ret,frame = cap.read()
        frame = cv2.resize(frame,(1280,720))
        
        #Recolor image
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # False as we can read as well as write w.r.t image
        
        # Make detections
        results =holistic.process(image)
        
        
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        if condition_0 == "Angle":
            if condition_1 == 0:
                    try: 
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                            shoulder_1 = [landmarks[cond_angle[0]].x,landmarks[cond_angle[0]].y]
                            elbow_1 = [landmarks[cond_angle[1]].x,landmarks[cond_angle[1]].y]
                            wrist_1 = [landmarks[cond_angle[2]].x,landmarks[cond_angle[2]].y]
                            
                            shoulder_2 = [landmarks[12].x,landmarks[12].y]
                            elbow_2 = [landmarks[14].x,landmarks[14].y]
                            wrist_2 = [landmarks[16].x,landmarks[16].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)
                            
                            per = np.interp(angle[0],(20,160),(100,0)) # LEFT
                            bar = np.interp(angle[0],(30,160),(600,200))
            
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            #print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Curl Counter Logic
                            color =(255,0,255)
                            if Signs[0]=="<" or Signs[1]==">":
                                if angle[0]>Angle_para[1]:
                                    stage = "Down"
                                    if per ==0:
                                        color =(0,255,0)
                                if angle[0]<Angle_para[0] and stage =="Down": # Count when we moves from down to up , >160 is full flat so down
                                    stage="Up"
                                    if per ==100:
                                        color=(0,255,0)
                                    counter+=1
                                    print(counter)
                            
                            
                            
            
                    except:
                         pass
                            
                            # Render curl counter and status box
                    cv2.rectangle(image,(5,5),(270,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
                            # Draw Bar
                    cv2.rectangle(image,(1000,200),(1055,600),color,3)
                    cv2.rectangle(image,(1000,int(bar)),(1055,600),color,cv2.FILLED)
                    cv2.putText(image,f'{int(per)}%',(1000,170),cv2.FONT_HERSHEY_PLAIN, 2,color, 3)
        
       
                            # Rep data
                    cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,str(counter),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                    cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,stage,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                    cv2.putText(image,Class[0],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                    if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                    if full_frame_res == 0:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 255, 0), 10) # green
                    else:
                                          cv2.rectangle(image, (0,0), (1280,697), (0, 0 ,255), 10) #red
                                    
    
            
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                            break
            elif condition_1==1:
                    try: 
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                            shoulder_1 = [landmarks[cond_angle[0]].x,landmarks[cond_angle[0]].y]
                            elbow_1 = [landmarks[cond_angle[1]].x,landmarks[cond_angle[1]].y]
                            wrist_1 = [landmarks[cond_angle[2]].x,landmarks[cond_angle[2]].y]
                            
                            shoulder_2 = [landmarks[cond_angle[3]].x,landmarks[cond_angle[3]].y]
                            elbow_2 = [landmarks[cond_angle[4]].x,landmarks[cond_angle[4]].y]
                            wrist_2 = [landmarks[cond_angle[5]].x,landmarks[cond_angle[5]].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)
                            
                            #per = np.interp(angle[0],(20,160),(100,0)) # LEFT
                            #bar = np.interp(angle[0],(30,160),(600,200))
            
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [800, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            #print(landmarks)
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Curl Counter Logic
                            color =(255,0,255)
                            if Signs[0]=="<" and Signs[1]==">":
                                # 2.) Oh- Press Curl Counter    
                                    if angle[0]<90:
                                           stage_o = "Down"
                                    if angle[0]>100 and stage_o=="Down":
                                            stage_o="Up"
                                            press+=1
                                            print(press)
                            
                    except:
                         pass
                            
                            # Render curl counter and status box
                    cv2.rectangle(image,(5,5),(270,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
                            # Draw Bar
                    #cv2.rectangle(image,(1000,200),(1055,600),color,3)
                    #cv2.rectangle(image,(1000,int(bar)),(1055,600),color,cv2.FILLED)
                    #cv2.putText(image,f'{int(per)}%',(1000,170),cv2.FONT_HERSHEY_PLAIN, 2,color, 3)
        
       
                            # Rep data
                    cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,str(press),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                    cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,stage_o,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                    cv2.putText(image,Class[1],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                    if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                                    if full_frame_res == 0:
                                          cv2.rectangle(image, (0,0), (800,720), (0, 255, 0), 10) # green
                                    else:
                                          cv2.rectangle(image, (0,0), (800,720), (0, 0 ,255), 10) #red
            
            
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                            break
        
                
                
            else:
                    cv2.putText(image,Class[2],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                           break
                
        #elif condition_0 ==:Angle_Ground
                   
    cap.release()
    cv2.destroyAllWindows()
                    

[x: 0.6580660939216614
y: 0.2342013716697693
z: -0.3393723964691162
visibility: 0.999901294708252
, x: 0.6703794002532959
y: 0.2009274959564209
z: -0.3344528377056122
visibility: 0.9999477863311768
, x: 0.6777178645133972
y: 0.1992843896150589
z: -0.3345388174057007
visibility: 0.9998770952224731
, x: 0.6858267784118652
y: 0.1978989988565445
z: -0.3345295488834381
visibility: 0.9999405145645142
, x: 0.6565619111061096
y: 0.2025555968284607
z: -0.30735212564468384
visibility: 0.999927282333374
, x: 0.6533481478691101
y: 0.20219188928604126
z: -0.307652086019516
visibility: 0.9998427629470825
, x: 0.6495715975761414
y: 0.20196270942687988
z: -0.3075887858867645
visibility: 0.9998964071273804
, x: 0.709846019744873
y: 0.21043027937412262
z: -0.24248352646827698
visibility: 0.9999194145202637
, x: 0.6614567041397095
y: 0.21503520011901855
z: -0.120566226541996
visibility: 0.9997619986534119
, x: 0.6763708591461182
y: 0.26476430892944336
z: -0.30288752913475037
visibility: 0.999987006187439

[x: 0.6522563695907593
y: 0.23891760408878326
z: -0.36611562967300415
visibility: 0.9998660087585449
, x: 0.6656867861747742
y: 0.2060338705778122
z: -0.3613094687461853
visibility: 0.99992436170578
, x: 0.672009289264679
y: 0.20516221225261688
z: -0.36140239238739014
visibility: 0.9998335838317871
, x: 0.6779802441596985
y: 0.2046070694923401
z: -0.3614021837711334
visibility: 0.9999174475669861
, x: 0.6518526673316956
y: 0.20684583485126495
z: -0.33405423164367676
visibility: 0.9998934268951416
, x: 0.648499608039856
y: 0.20686030387878418
z: -0.33441799879074097
visibility: 0.999775767326355
, x: 0.645045280456543
y: 0.20697569847106934
z: -0.334353506565094
visibility: 0.9998409748077393
, x: 0.7036991715431213
y: 0.21574968099594116
z: -0.2533918023109436
visibility: 0.9998966455459595
, x: 0.6593382358551025
y: 0.21812854707241058
z: -0.1206069141626358
visibility: 0.999689519405365
, x: 0.6671441793441772
y: 0.2709940969944
z: -0.32055625319480896
visibility: 0.9999819993972778


[x: 0.635953962802887
y: 0.24473345279693604
z: -0.3838108777999878
visibility: 0.9998941421508789
, x: 0.6499816179275513
y: 0.21124504506587982
z: -0.38366931676864624
visibility: 0.9999305605888367
, x: 0.6564478874206543
y: 0.21002539992332458
z: -0.38346344232559204
visibility: 0.9998666048049927
, x: 0.6633777618408203
y: 0.20893914997577667
z: -0.3835647702217102
visibility: 0.9999247193336487
, x: 0.6350909471511841
y: 0.21360287070274353
z: -0.3527548909187317
visibility: 0.9999020099639893
, x: 0.6312683820724487
y: 0.21398614346981049
z: -0.35313713550567627
visibility: 0.9998182654380798
, x: 0.6271755695343018
y: 0.21452109515666962
z: -0.3529369831085205
visibility: 0.9998512864112854
, x: 0.6877340078353882
y: 0.21854083240032196
z: -0.2742542326450348
visibility: 0.9999178051948547
, x: 0.6416999697685242
y: 0.22477486729621887
z: -0.1462801992893219
visibility: 0.9997546076774597
, x: 0.6524208188056946
y: 0.2779560983181
z: -0.3454836308956146
visibility: 0.9999814629

[x: 0.6311495900154114
y: 0.25078701972961426
z: -0.3235469460487366
visibility: 0.9999188184738159
, x: 0.6444065570831299
y: 0.2171511948108673
z: -0.3185381293296814
visibility: 0.9999382495880127
, x: 0.6509506106376648
y: 0.21560503542423248
z: -0.31852811574935913
visibility: 0.9998955726623535
, x: 0.6576026082038879
y: 0.21445797383785248
z: -0.31853795051574707
visibility: 0.9999338388442993
, x: 0.6293166875839233
y: 0.21961498260498047
z: -0.28901731967926025
visibility: 0.9999134540557861
, x: 0.6256304979324341
y: 0.21998435258865356
z: -0.2892983853816986
visibility: 0.9998574256896973
, x: 0.621519148349762
y: 0.22030864655971527
z: -0.2892431914806366
visibility: 0.9998683333396912
, x: 0.6828750967979431
y: 0.22428928315639496
z: -0.21627648174762726
visibility: 0.9999357461929321
, x: 0.6352336406707764
y: 0.22932961583137512
z: -0.08117422461509705
visibility: 0.9998093843460083
, x: 0.647630512714386
y: 0.2828514873981476
z: -0.2824821472167969
visibility: 0.9999815

[x: 0.6323730945587158
y: 0.2510231137275696
z: -0.3453735411167145
visibility: 0.9999212622642517
, x: 0.646210789680481
y: 0.21767255663871765
z: -0.3417852520942688
visibility: 0.9999417662620544
, x: 0.6526045203208923
y: 0.2167360931634903
z: -0.34184977412223816
visibility: 0.9999009966850281
, x: 0.6595640778541565
y: 0.21598991751670837
z: -0.34186652302742004
visibility: 0.9999383091926575
, x: 0.6308015584945679
y: 0.21912437677383423
z: -0.3101639151573181
visibility: 0.9999160766601562
, x: 0.6268060803413391
y: 0.21930241584777832
z: -0.3104800581932068
visibility: 0.9998598694801331
, x: 0.6227684617042542
y: 0.2196093499660492
z: -0.3104243874549866
visibility: 0.9998686909675598
, x: 0.6851484179496765
y: 0.2286025732755661
z: -0.23767586052417755
visibility: 0.9999400973320007
, x: 0.6350975632667542
y: 0.2297249287366867
z: -0.09825275093317032
visibility: 0.9998111128807068
, x: 0.6483301520347595
y: 0.2838013470172882
z: -0.30285733938217163
visibility: 0.9999823570

...

[x: 0.61341392993927
y: 0.24645806849002838
z: -0.2862127125263214
visibility: 0.9998643398284912
, x: 0.6294337511062622
y: 0.21457627415657043
z: -0.2857343554496765
visibility: 0.9999421834945679
, x: 0.6366558074951172
y: 0.21434834599494934
z: -0.28565356135368347
visibility: 0.9997972846031189
, x: 0.6448385715484619
y: 0.214432954788208
z: -0.2856719493865967
visibility: 0.9999462366104126
, x: 0.6153613924980164
y: 0.21401718258857727
z: -0.2500917315483093
visibility: 0.9999279975891113
, x: 0.6121934056282043
y: 0.21302111446857452
z: -0.25026652216911316
visibility: 0.9997214674949646
, x: 0.6092708110809326
y: 0.21198637783527374
z: -0.2502259612083435
visibility: 0.9998829364776611
, x: 0.6686346530914307
y: 0.23168396949768066
z: -0.19938191771507263
visibility: 0.9998618960380554
, x: 0.6247719526290894
y: 0.22191564738750458
z: -0.06389904767274857
visibility: 0.9994822144508362
, x: 0.6296151280403137
y: 0.27868950366973877
z: -0.2486189901828766
visibility: 0.99998134

[x: 0.6060214042663574
y: 0.2496713399887085
z: -0.33918821811676025
visibility: 0.9999008178710938
, x: 0.6221229434013367
y: 0.21682648360729218
z: -0.3383462727069855
visibility: 0.999952495098114
, x: 0.6283518075942993
y: 0.21653954684734344
z: -0.3382658362388611
visibility: 0.9998549222946167
, x: 0.6348651647567749
y: 0.2164573073387146
z: -0.3382732570171356
visibility: 0.9999551177024841
, x: 0.6086523532867432
y: 0.21653872728347778
z: -0.3006380498409271
visibility: 0.9999378323554993
, x: 0.6055464148521423
y: 0.2158551961183548
z: -0.30095747113227844
visibility: 0.9997962117195129
, x: 0.6024068593978882
y: 0.21532130241394043
z: -0.3008551597595215
visibility: 0.999894917011261
, x: 0.6618459224700928
y: 0.23157475888729095
z: -0.24808232486248016
visibility: 0.9999018907546997
, x: 0.620120644569397
y: 0.2257077395915985
z: -0.08079599589109421
visibility: 0.9996352791786194
, x: 0.6213369965553284
y: 0.2839932441711426
z: -0.3019961714744568
visibility: 0.999983251094

[x: 0.5980348587036133
y: 0.24652500450611115
z: -0.2876361012458801
visibility: 0.9999315738677979
, x: 0.6133275032043457
y: 0.21549803018569946
z: -0.283549040555954
visibility: 0.999962568283081
, x: 0.6200357675552368
y: 0.21544311940670013
z: -0.28347715735435486
visibility: 0.9999002814292908
, x: 0.6263979077339172
y: 0.21540768444538116
z: -0.28350239992141724
visibility: 0.9999642372131348
, x: 0.5993954539299011
y: 0.2152513563632965
z: -0.2503548562526703
visibility: 0.9999499320983887
, x: 0.5960591435432434
y: 0.2149261236190796
z: -0.25062379240989685
visibility: 0.9998589754104614
, x: 0.5927960872650146
y: 0.21472786366939545
z: -0.25050610303878784
visibility: 0.9999138116836548
, x: 0.6522281169891357
y: 0.2296954095363617
z: -0.19431070983409882
visibility: 0.9999327063560486
, x: 0.6073001623153687
y: 0.2256237268447876
z: -0.043472770601511
visibility: 0.999751091003418
, x: 0.6135573387145996
y: 0.28047388792037964
z: -0.25205475091934204
visibility: 0.9999854564

[x: 0.5963034629821777
y: 0.24593855440616608
z: -0.30834275484085083
visibility: 0.9999512434005737
, x: 0.61170494556427
y: 0.21510279178619385
z: -0.30759087204933167
visibility: 0.9999690651893616
, x: 0.6183909177780151
y: 0.2150464653968811
z: -0.3075104057788849
visibility: 0.9999293684959412
, x: 0.6246463060379028
y: 0.21499381959438324
z: -0.3075319230556488
visibility: 0.9999701380729675
, x: 0.598164975643158
y: 0.2148926854133606
z: -0.27381229400634766
visibility: 0.9999581575393677
, x: 0.5948683023452759
y: 0.21463091671466827
z: -0.2741173803806305
visibility: 0.9998993277549744
, x: 0.5916754007339478
y: 0.2144341766834259
z: -0.27395063638687134
visibility: 0.9999275207519531
, x: 0.6506866216659546
y: 0.2294357270002365
z: -0.21908342838287354
visibility: 0.9999525547027588
, x: 0.6072250008583069
y: 0.22559456527233124
z: -0.0642116367816925
visibility: 0.9998268485069275
, x: 0.6115133762359619
y: 0.2798151969909668
z: -0.2725481390953064
visibility: 0.99998700618

[x: 0.5963560342788696
y: 0.24557043612003326
z: -0.2605162560939789
visibility: 0.9999624490737915
, x: 0.6121033430099487
y: 0.2145058959722519
z: -0.25495386123657227
visibility: 0.9999744892120361
, x: 0.6186909079551697
y: 0.21444487571716309
z: -0.25486522912979126
visibility: 0.9999460577964783
, x: 0.6250436902046204
y: 0.21459437906742096
z: -0.2549445331096649
visibility: 0.9999755024909973
, x: 0.5986636877059937
y: 0.21404007077217102
z: -0.2255888283252716
visibility: 0.9999650716781616
, x: 0.59540194272995
y: 0.21373394131660461
z: -0.2258319854736328
visibility: 0.999921977519989
, x: 0.5922046899795532
y: 0.21332430839538574
z: -0.2256651222705841
visibility: 0.9999386072158813
, x: 0.6509784460067749
y: 0.22989895939826965
z: -0.15284433960914612
visibility: 0.9999650716781616
, x: 0.6077895164489746
y: 0.22484850883483887
z: -0.02179659530520439
visibility: 0.9998695850372314
, x: 0.6112382411956787
y: 0.27982378005981445
z: -0.22087065875530243
visibility: 0.9999895

In [10]:
Class = ['Single_arm_Dumbell','OH-Press','Neutral'] # Add on the Exercise names in this List
cap = cv2.VideoCapture(r"C:\Users\KUNAL\Documents\Pose_Detection\combined\1.mp4")
## Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic: #
    while cap.isOpened():
        timer=cv2.getTickCount()
        ret,frame = cap.read()
        frame = cv2.resize(frame,(1280,720))
        
        #Recolor image
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # False as we can read as well as write w.r.t image
        
        # Make detections
        results =holistic.process(image)
        
        
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        if condition_1 == 1:
                    try: 
                            landmarks = results.pose_landmarks.landmark
                            print(landmarks)
                            shoulder_1 = [landmarks[cond_angle[0]].x,landmarks[cond_angle[0]].y]
                            elbow_1 = [landmarks[cond_angle[1]].x,landmarks[cond_angle[1]].y]
                            wrist_1 = [landmarks[cond_angle[2]].x,landmarks[cond_angle[2]].y]
                            
                            shoulder_2 = [landmarks[12].x,landmarks[12].y]
                            elbow_2 = [landmarks[14].x,landmarks[14].y]
                            wrist_2 = [landmarks[16].x,landmarks[16].y]
                            
                            angle = calculate_angle(shoulder_1, elbow_1, wrist_1,shoulder_2, elbow_2, wrist_2)
                            
                            #per = np.interp(angle[0],(20,160),(100,0)) # LEFT
                            #bar = np.interp(angle[0],(30,160),(600,200))
            
                            cv2.putText(image, str(round(angle[0],2)), 
                            tuple(np.multiply(elbow_1, [800, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                            cv2.putText(image, str(round(angle[1],2)), 
                            tuple(np.multiply(elbow_1, [1280, 720]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                            
                            
                            
                            #fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) # Get frames per second
                            # Curl Counter Logic
                            color =(255,0,255)
                            if Signs[0]=="<" and Signs[1]==">":
                                # 2.) Oh- Press Curl Counter    
                                    if angle[0]<90:
                                           stage_o = "Down"
                                    if angle[0]>100 and stage_o=="Down":
                                            stage_o="Up"
                                            press+=1
                                            print(press)
                            
                            
                    except:
                         pass
                            
                            # Render curl counter and status box
                    cv2.rectangle(image,(5,5),(270,80),(245,117,66),-1)
        
                            # fps
                            #cv2.putText(image,str(fps),(90,19),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
                            # Draw Bar
                    #cv2.rectangle(image,(1000,200),(1055,600),color,3)
                    #cv2.rectangle(image,(1000,int(bar)),(1055,600),color,cv2.FILLED)
                    #cv2.putText(image,f'{int(per)}%',(1000,170),cv2.FONT_HERSHEY_PLAIN, 2,color, 3)
        
       
                            # Rep data
                    cv2.putText(image,'REPS',(17,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,str(press),(20,72),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
                            #Stage Data
                    cv2.putText(image,'STAGE',(120,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image,stage_o,(120,65),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                            # Render Class Name
                    cv2.putText(image,Class[1],(500,45),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
               
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )   
                            
                    if results.pose_landmarks !=None:
                                    face_res, pose_res, full_frame_res, angle_res, elbow_flat_angle, elbow_straight_angle, hand_straight_angle = pose_estimate(
                                    results.pose_landmarks, image)
                                    if full_frame_res == 1:
                                          cv2.rectangle(image, (0,0), (800,720), (0, 255, 0), 10) # green
                                    else:
                                          cv2.rectangle(image, (0,0), (800,720), (0, 0 ,255), 10) #red
            
            
                    cv2.imshow("Mediapipe Feed",image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                            break
        
    cap.release()
    cv2.destroyAllWindows()
                

[x: 0.6586964130401611
y: 0.40754014253616333
z: 0.008961968123912811
visibility: 0.9895089864730835
, x: 0.6736438870429993
y: 0.384377658367157
z: -0.017080241814255714
visibility: 0.9932436347007751
, x: 0.6695959568023682
y: 0.3963150978088379
z: -0.017056802287697792
visibility: 0.9888361096382141
, x: 0.6687736511230469
y: 0.3941158354282379
z: -0.017032558098435402
visibility: 0.9935265779495239
, x: 0.6805647611618042
y: 0.40383380651474
z: -0.0033670454286038876
visibility: 0.9941493272781372
, x: 0.6804169416427612
y: 0.4047031104564667
z: -0.0034572456497699022
visibility: 0.9917876124382019
, x: 0.6803677082061768
y: 0.40555140376091003
z: -0.0034733260981738567
visibility: 0.9947385191917419
, x: 0.6595090627670288
y: 0.4057471752166748
z: -0.09026427567005157
visibility: 0.9898635745048523
, x: 0.6801583766937256
y: 0.4199581444263458
z: -0.028834262862801552
visibility: 0.9943630695343018
, x: 0.644412100315094
y: 0.41613608598709106
z: -0.012214572168886662
visibility: 

[x: 0.6025363206863403
y: 0.37436938285827637
z: -0.09121128916740417
visibility: 0.9880311489105225
, x: 0.6275274157524109
y: 0.3715662956237793
z: -0.11751929670572281
visibility: 0.9934833645820618
, x: 0.6283693909645081
y: 0.3716168701648712
z: -0.11756017059087753
visibility: 0.987812340259552
, x: 0.6295111179351807
y: 0.37073466181755066
z: -0.11762712895870209
visibility: 0.993837833404541
, x: 0.6246848702430725
y: 0.3688032329082489
z: -0.08518097549676895
visibility: 0.993969202041626
, x: 0.6273530125617981
y: 0.3678942322731018
z: -0.08531235158443451
visibility: 0.9897799491882324
, x: 0.6284120082855225
y: 0.3669065237045288
z: -0.0854160264134407
visibility: 0.994718074798584
, x: 0.635916531085968
y: 0.3810402750968933
z: -0.18614336848258972
visibility: 0.9889969825744629
, x: 0.6349226236343384
y: 0.3896184265613556
z: -0.028225962072610855
visibility: 0.993584156036377
, x: 0.593928337097168
y: 0.4037635326385498
z: -0.10925404727458954
visibility: 0.9890715479850

]
[x: 0.7021992802619934
y: 0.20739233493804932
z: -0.02995506301522255
visibility: 0.9442617297172546
, x: 0.717103123664856
y: 0.20730650424957275
z: -0.061723459511995316
visibility: 0.9708505272865295
, x: 0.7181158661842346
y: 0.20747911930084229
z: -0.06179909035563469
visibility: 0.9397588968276978
, x: 0.7185489535331726
y: 0.20775598287582397
z: -0.06186113879084587
visibility: 0.9741363525390625
, x: 0.7169549465179443
y: 0.20820772647857666
z: -0.024543479084968567
visibility: 0.9713281989097595
, x: 0.7180119752883911
y: 0.20902268588542938
z: -0.024745723232626915
visibility: 0.9471460580825806
, x: 0.7184498310089111
y: 0.2098146677017212
z: -0.02496049739420414
visibility: 0.978901743888855
, x: 0.7315847277641296
y: 0.2342437505722046
z: -0.1445438414812088
visibility: 0.9529348611831665
, x: 0.7288304567337036
y: 0.22945839166641235
z: 0.023105444386601448
visibility: 0.9747844934463501
, x: 0.7047260403633118
y: 0.22278624773025513
z: -0.05459677428007126
visibility: 

]
[x: 0.7516717314720154
y: 0.35949453711509705
z: 0.024357866495847702
visibility: 0.947645366191864
, x: 0.7595155835151672
y: 0.3527054488658905
z: -0.008167356252670288
visibility: 0.9713999032974243
, x: 0.7592354416847229
y: 0.3523571789264679
z: -0.008187985047698021
visibility: 0.9421696662902832
, x: 0.759413480758667
y: 0.35220083594322205
z: -0.008190189488232136
visibility: 0.9742761850357056
, x: 0.7595885992050171
y: 0.353546142578125
z: 0.001851050416007638
visibility: 0.9729556441307068
, x: 0.7591964602470398
y: 0.35322004556655884
z: 0.0017107513267546892
visibility: 0.9505091309547424
, x: 0.7587843537330627
y: 0.3528937101364136
z: 0.0018217662582173944
visibility: 0.9795867204666138
, x: 0.7534139156341553
y: 0.34883877635002136
z: -0.006070706993341446
visibility: 0.9511186480522156
, x: 0.7528676986694336
y: 0.35613518953323364
z: -0.014411699958145618
visibility: 0.9735949039459229
, x: 0.7377272844314575
y: 0.3636626899242401
z: 0.0007984411204233766
visibility

]
[x: 0.6454141139984131
y: 0.5574355125427246
z: -0.0711890384554863
visibility: 0.9632185697555542
, x: 0.6730872988700867
y: 0.5756626129150391
z: -0.08553548902273178
visibility: 0.9795947074890137
, x: 0.6753756999969482
y: 0.5918549299240112
z: -0.08568980544805527
visibility: 0.9588959217071533
, x: 0.6684305667877197
y: 0.5956294536590576
z: -0.08569086343050003
visibility: 0.9813438653945923
, x: 0.6612482666969299
y: 0.5742239952087402
z: -0.05482562631368637
visibility: 0.9806388020515442
, x: 0.6620187759399414
y: 0.573514997959137
z: -0.05508563667535782
visibility: 0.9650568962097168
, x: 0.6626508831977844
y: 0.57262122631073
z: -0.05503937602043152
visibility: 0.9853960275650024
, x: 0.6656857132911682
y: 0.6205549240112305
z: -0.11487305164337158
visibility: 0.9641671180725098
, x: 0.6585623025894165
y: 0.6006850004196167
z: 0.018120791763067245
visibility: 0.9808256030082703
, x: 0.6296810507774353
y: 0.5765683054924011
z: -0.08089244365692139
visibility: 0.9795194864

...

[x: 0.3220054507255554
y: 0.4269470274448395
z: 0.1612972915172577
visibility: 0.9857640266418457
, x: 0.3219873011112213
y: 0.4196952283382416
z: 0.19955535233020782
visibility: 0.9929359555244446
, x: 0.3275502026081085
y: 0.4167671501636505
z: 0.1994556188583374
visibility: 0.9857675433158875
, x: 0.32845553755760193
y: 0.41658473014831543
z: 0.1994057595729828
visibility: 0.9933910369873047
, x: 0.3152690529823303
y: 0.43167757987976074
z: 0.18601951003074646
visibility: 0.9930468797683716
, x: 0.30450209975242615
y: 0.4351132810115814
z: 0.1857929527759552
visibility: 0.9875744581222534
, x: 0.30601102113723755
y: 0.4368329644203186
z: 0.18605366349220276
visibility: 0.9944277405738831
, x: 0.33187127113342285
y: 0.4467909038066864
z: 0.3019469380378723
visibility: 0.9879945516586304
, x: 0.30366626381874084
y: 0.46307143568992615
z: 0.2212035357952118
visibility: 0.9916675686836243
, x: 0.3376283347606659
y: 0.4456329345703125
z: 0.1925865113735199
visibility: 0.9900475144386292


]
[x: 0.34181109070777893
y: 0.4184079170227051
z: -0.08321617543697357
visibility: 0.9887195229530334
, x: 0.33162829279899597
y: 0.41452455520629883
z: -0.07725924253463745
visibility: 0.9950294494628906
, x: 0.3357796370983124
y: 0.4135541319847107
z: -0.07738257199525833
visibility: 0.9901097416877747
, x: 0.3378060758113861
y: 0.41342976689338684
z: -0.07772878557443619
visibility: 0.9952673316001892
, x: 0.32625848054885864
y: 0.42138317227363586
z: -0.09000904113054276
visibility: 0.9950053095817566
, x: 0.3261292278766632
y: 0.42624422907829285
z: -0.09013910591602325
visibility: 0.9911450147628784
, x: 0.32545262575149536
y: 0.43182113766670227
z: -0.09008032083511353
visibility: 0.9957749843597412
, x: 0.3388040065765381
y: 0.44663330912590027
z: -0.02627735771238804
visibility: 0.9921337962150574
, x: 0.33193641901016235
y: 0.46978798508644104
z: -0.08777743577957153
visibility: 0.994113028049469
, x: 0.34553128480911255
y: 0.4408230185508728
z: -0.06094229966402054
visibili

]
[x: 0.3278183341026306
y: 0.4453032910823822
z: -0.29261356592178345
visibility: 0.9723764657974243
, x: 0.3327818214893341
y: 0.45794999599456787
z: -0.3132394254207611
visibility: 0.9898785948753357
, x: 0.332585871219635
y: 0.45974594354629517
z: -0.31333786249160767
visibility: 0.98298579454422
, x: 0.3284706473350525
y: 0.4648863971233368
z: -0.31370991468429565
visibility: 0.9902936816215515
, x: 0.32879403233528137
y: 0.4564708173274994
z: -0.32004424929618835
visibility: 0.9877236485481262
, x: 0.3418870270252228
y: 0.46863555908203125
z: -0.3201574683189392
visibility: 0.9817803502082825
, x: 0.3415646553039551
y: 0.47020232677459717
z: -0.3202759325504303
visibility: 0.9875436425209045
, x: 0.31998488306999207
y: 0.4882822632789612
z: -0.3072735667228699
visibility: 0.9901059865951538
, x: 0.32860809564590454
y: 0.4825406074523926
z: -0.3408413529396057
visibility: 0.9893667697906494
, x: 0.3254000246524811
y: 0.482543021440506
z: -0.27824512124061584
visibility: 0.98892176

[x: 0.30575257539749146
y: 0.45990341901779175
z: 0.17823243141174316
visibility: 0.9649056196212769
, x: 0.3069537281990051
y: 0.4589831531047821
z: 0.18554213643074036
visibility: 0.9877051711082458
, x: 0.3075655400753021
y: 0.4606170952320099
z: 0.18535122275352478
visibility: 0.9791436791419983
, x: 0.30750709772109985
y: 0.46340009570121765
z: 0.1849544644355774
visibility: 0.9877108931541443
, x: 0.30518245697021484
y: 0.45990821719169617
z: 0.18347403407096863
visibility: 0.9856747388839722
, x: 0.3052442669868469
y: 0.4641208350658417
z: 0.18335004150867462
visibility: 0.9791024923324585
, x: 0.304731547832489
y: 0.4672745168209076
z: 0.18341046571731567
visibility: 0.985845148563385
, x: 0.3067858815193176
y: 0.505142867565155
z: 0.19451332092285156
visibility: 0.9891514182090759
, x: 0.2926066517829895
y: 0.48208707571029663
z: 0.17933200299739838
visibility: 0.9880730509757996
, x: 0.30033910274505615
y: 0.4756418466567993
z: 0.17849385738372803
visibility: 0.98714256286621

[x: 0.36163821816444397
y: 0.6074081659317017
z: 0.15341632068157196
visibility: 0.7689213156700134
, x: 0.36022791266441345
y: 0.6211260557174683
z: 0.12595108151435852
visibility: 0.8450193405151367
, x: 0.3575192093849182
y: 0.6204862594604492
z: 0.12587130069732666
visibility: 0.810784101486206
, x: 0.34406667947769165
y: 0.6202534437179565
z: 0.1255311667919159
visibility: 0.8412782549858093
, x: 0.3685693144798279
y: 0.6251809597015381
z: 0.12939678132534027
visibility: 0.8472122550010681
, x: 0.37352055311203003
y: 0.6267287135124207
z: 0.1293930858373642
visibility: 0.8378932476043701
, x: 0.3777148127555847
y: 0.6287686824798584
z: 0.12949153780937195
visibility: 0.8658329844474792
, x: 0.3174336552619934
y: 0.6294629573822021
z: -0.010088612325489521
visibility: 0.8540228605270386
, x: 0.3789929151535034
y: 0.6436918377876282
z: 0.03302004560828209
visibility: 0.8974090218544006
, x: 0.3424510657787323
y: 0.6073426604270935
z: 0.12183736264705658
visibility: 0.903729021549224